# FITS visualisation using Jupyter

### Interactive Visualisation using WebRTC and H.264 Encoding

In [ ]:
import cv2
import asyncio
import uvloop
import concurrent
import numpy as np
import bqplot.pyplot as plt
from pyastrovis.webrtc import WebRTCStream
from pyastrovis.widget import WebRTCWidget
from pyastrovis.fits import FITSImageCubeStream
from ipywidgets import IntSlider, HBox, VBox, Layout
from IPython.display import display

uvloop.install()

In [ ]:
server = await WebRTCWidget.create_server(host='0.0.0.0', port=8080)

In [ ]:
fits_file = './ngc6946.fits'
fits = await FITSImageCubeStream.open(fits_file, num_processes=4)

In [ ]:
panel = server.create_panel(320, 320)
x = [i for i in range(fits.get_num_channels())]
y = []
fig = plt.figure()
fig.layout.height = '280px'
fig.layout.width = '668px'
axes_options = {'x': {'label': 'Channel'}, 'y': {'label': 'Jy/beam'}}
lines = plt.plot(x=x, y=y, colors=['red'], axes_options=axes_options)

In [ ]:
loop = asyncio.get_running_loop()
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

def modify(data):
    img = 255*((data-data.min())/data.ptp())
    img = img.astype(np.uint8)
    return cv2.applyColorMap(img, cv2.COLORMAP_HSV)
    
async def channel():
    while True:
        value = await wait_for_change(slider, 'value')
        data = await fits.get_channel_data(value)
        img = await loop.run_in_executor(None, modify, data)
        await panel.add_data(img, 'rgb24')

async def spectra():
    while True:
        x, y = await wait_for_change(panel, 'position')
        spec = await fits.get_spectral_line(x, y)
        lines.y = spec
        
asyncio.ensure_future(channel())
asyncio.ensure_future(spectra())
slider = IntSlider(description='Channel:', layout=Layout(width='100%'), 
                   value=0, min=0, step=2, max=fits.get_num_channels()-1)
display(HBox([panel, fig]), slider)

In [ ]:
await panel.close()
await server.close()
await fits.close()

### Image Smoothing

In [ ]:
%matplotlib inline
import cv2
import warnings
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from pyastrovis.fits import FITSImageCubeStream
warnings.simplefilter(action='ignore', category=FutureWarning)

fits_file = '/Users/dpallot/Projects/jacal/summit_demo/oskar/ingest/demo_f.all.fits'
fits = await FITSImageCubeStream.open(fits_file)
img = await fits.get_channel_data(1090)

In [ ]:
image1 = ndimage.gaussian_filter(img, 10)
fig, ax = plt.subplots(1, 2, figsize=(15,15))
ax[0].imshow(img)
ax[1].imshow(image1)
plt.show()

### Volumentric Rendering

In [ ]:
import warnings
import ipyvolume as ipv
import numpy as np
import scipy.ndimage
from pyastrovis.fits import FITSImageCubeStream
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
fits_file = '/Users/dpallot/Projects/jacal/summit_demo/oskar/ingest/demo_f.all.fits'
fits = await FITSImageCubeStream.open(fits_file)

In [ ]:
channels = [await fits.get_channel_data(i) for i in range(1070, 1135)]
cube = scipy.ndimage.gaussian_filter(np.dstack(channels), 2)

In [ ]:
ipv.quickvolshow(cube, lighting=True, level=[0.29, 0.3, 0.33], 
                 opacity=[0.2, 0.2, 0.2], level_width=0.05)

### Source Detection

In [ ]:
from astropy.stats import sigma_clipped_stats
from photutils import datasets
from photutils import DAOStarFinder
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import CircularAperture

data = await fits.get_channel_data(1111)
data = ndimage.gaussian_filter(data, 10)
mean, median, std = sigma_clipped_stats(data, sigma=1.0)
daofind = DAOStarFinder(fwhm=3.0, threshold=1.2*std)    
sources = daofind(data - median)  
positions = (sources['xcentroid'], sources['ycentroid'])
apertures = CircularAperture(positions, r=100.)

fig, ax = plt.subplots(1, 2, figsize=(15,15))
ax[0].imshow(data, origin='lower')
ax[1].imshow(data, cmap='Greys', origin='lower')
apertures.plot(color='blue', lw=1.5, alpha=0.5)
plt.show()